In [1]:
import cv2
import matplotlib.pyplot as plt
import os
import xml.etree.ElementTree as ET
import glob
import numpy as np

In [2]:
work_dir = os.getcwd()
output_path = work_dir +'/Label/'

S_path = work_dir + '/Seg_images/'

if not os.path.exists(output_path):
    os.makedirs(output_path)
    
dir_path= os.path.join(work_dir, 'Images/')

In [3]:
def _create_object_subtree(annotation, label, xmin, ymin, xmax, ymax):
    
    """ Create the sub-tree related to a given object and update the root """
    _object = ET.SubElement(annotation, "object")
    ET.SubElement(_object, "name").text = str(label)
    #ET.SubElement(_object, "radiologist").text = obj[2]
    ET.SubElement(_object, "pose").text = "Unspecified"
    ET.SubElement(_object, "truncated").text = "0"
    ET.SubElement(_object, "difficult").text = "0"

    _bndbox = ET.SubElement(_object, "bndbox")
    ET.SubElement(_bndbox, "xmin").text = str(xmin)
    ET.SubElement(_bndbox, "ymin").text = str(ymin)
    ET.SubElement(_bndbox, "xmax").text = str(xmax)
    ET.SubElement(_bndbox, "ymax").text = str(ymax)
    return annotation

In [4]:
def _Add_box(annotation,gray_img,Label):
    # Convert the grayscale image to binary
    ret, binary = cv2.threshold(gray_img, 100, 255, cv2.THRESH_OTSU)
    inverted_binary = ~binary
    contours, hierarchy = cv2.findContours(inverted_binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Show the total number of contours that were detected
    #print('contours detected: ' + str(len(contours)))
    # Draw a bounding box around all contours
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        #print(x,y,w,h)
        if x!=0 and y!=0:
            Padding=15
            _create_object_subtree(annotation, Label , (x-Padding), (y- Padding) , (x+w + Padding), (y+h +Padding ))
    return annotation


In [5]:
def _read_mulit_image(img_path,N_path):
    meta = cv2.imread(img_path, cv2.COLOR_BGR2GRAY)
    image_id = os.path.basename(img_path)
    basename_no_ext = os.path.splitext(image_id)[0]
    Ma_imge = os.path.join(N_path, str(basename_no_ext + '_MA.tif'))
    He_imge = os.path.join(N_path, str(basename_no_ext + '_HE.tif'))
    Ex_imge = os.path.join(N_path, str(basename_no_ext + '_EX.tif'))
    Se_imge = os.path.join(N_path, str(basename_no_ext + '_SE.tif'))
    Od_imge = os.path.join(N_path, str(basename_no_ext + '_OD.tif'))
    if os.path.exists(Ma_imge):
        MA = cv2.imread(Ma_imge, cv2.COLOR_BGR2GRAY)
    else:
        MA = np.array('0')
    if os.path.exists(He_imge):
        HE = cv2.imread(He_imge, cv2.COLOR_BGR2GRAY)
    else:
        HE =  np.array('0')
    if os.path.exists(Ex_imge):
        EX = cv2.imread(Ex_imge, cv2.COLOR_BGR2GRAY)
    else:
        EX =  np.array('0')
    if os.path.exists(Se_imge):
        SE = cv2.imread(Se_imge, cv2.COLOR_BGR2GRAY)
    else:
        SE =  np.array('0')
    if os.path.exists(Od_imge):
        OD = cv2.imread(Od_imge, cv2.COLOR_BGR2GRAY)
    else:
        OD =  np.array('0')
    return meta, image_id, basename_no_ext , MA, HE, EX, SE, OD
    
    
    

In [6]:
for img_path in glob.glob(dir_path + '/*.jpg'):
  annotation = ET.Element('annotation')
  meta, image_id, basename_no_ext , MA, HE, EX, SE, OD = _read_mulit_image(img_path,S_path)
  # ##### Beginning of XML #####

  # File Sub-Elements
  ET.SubElement(annotation, "folder").text="train"
  ET.SubElement(annotation, "filename").text=str(image_id)
  ET.SubElement(annotation, "path").text=str(img_path)

  # SRC Sub-Element
  _src = ET.SubElement(annotation, "src")
  ET.SubElement(_src, "database").text="train"

  # Size Sub-Element
  _size = ET.SubElement(annotation, "size")
  ET.SubElement(_size, "width").text=str(meta.shape[1])
  ET.SubElement(_size, "height").text=str(meta.shape[0])
  ET.SubElement(_size, "depth").text=str('3')

  # Segmented Sub-Element
  ET.SubElement(annotation, "segmented").text="0"

  print(image_id)
  if MA.size != 1 : 
    _Add_box(annotation,MA,"Micro") 
  if HE.size != 1 : 
    _Add_box(annotation,HE,"Ham") 
  if EX.size != 1 : 
    _Add_box(annotation,EX,"HEx")
  if SE.size != 1 : 
    _Add_box(annotation,SE,"SEx")
  if OD.size != 1 : 
    _Add_box(annotation,OD,"OD")
  

  # Save to output directory
  ET.ElementTree(annotation).write(os.path.join(output_path, basename_no_ext+'.xml'))

IDRiD_65.jpg
IDRiD_47.jpg
IDRiD_51.jpg
IDRiD_50.jpg
IDRiD_24.jpg
IDRiD_66.jpg
IDRiD_21.jpg
IDRiD_23.jpg
IDRiD_74.jpg
IDRiD_22.jpg
IDRiD_16.jpg
IDRiD_55.jpg
IDRiD_14.jpg
IDRiD_48.jpg
IDRiD_42.jpg
IDRiD_27.jpg
IDRiD_30.jpg
IDRiD_04.jpg
IDRiD_59.jpg
IDRiD_58.jpg
IDRiD_28.jpg
IDRiD_35.jpg
IDRiD_09.jpg
IDRiD_57.jpg
IDRiD_31.jpg
IDRiD_75.jpg
IDRiD_17.jpg
IDRiD_69.jpg
IDRiD_77.jpg
IDRiD_54.jpg
IDRiD_62.jpg
IDRiD_39.jpg
IDRiD_03.jpg
IDRiD_71.jpg
IDRiD_10.jpg
IDRiD_80.jpg
IDRiD_05.jpg
IDRiD_43.jpg
IDRiD_63.jpg
IDRiD_79.jpg
IDRiD_49.jpg
IDRiD_36.jpg
IDRiD_13.jpg
IDRiD_72.jpg
IDRiD_52.jpg
IDRiD_11.jpg
IDRiD_12.jpg
IDRiD_68.jpg
IDRiD_08.jpg
IDRiD_18.jpg
IDRiD_60.jpg
IDRiD_38.jpg
IDRiD_53.jpg
IDRiD_46.jpg
IDRiD_32.jpg
IDRiD_64.jpg
IDRiD_44.jpg
IDRiD_45.jpg
IDRiD_33.jpg
IDRiD_56.jpg
IDRiD_07.jpg
IDRiD_02.jpg
IDRiD_37.jpg
IDRiD_19.jpg
IDRiD_26.jpg
IDRiD_01.jpg
IDRiD_70.jpg
IDRiD_15.jpg
IDRiD_29.jpg
IDRiD_73.jpg
IDRiD_61.jpg
IDRiD_06.jpg
IDRiD_41.jpg
IDRiD_40.jpg
IDRiD_78.jpg
IDRiD_76.jpg
IDRiD_20.jpg